In [13]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import keras
import os
import requests
import shutil

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, GlobalAvgPool2D, BatchNormalization, add, Input
from keras.utils import to_categorical
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
#from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from io import open
from zipfile import ZipFile
from keras.models import Model
from tensorflow.python.keras.preprocessing import image


In [14]:
os.listdir()

['.ipynb_checkpoints',
 'interference-image.jpg',
 'Rex the robot-Copy1.ipynb',
 'Rex the robot.ipynb',
 'Rex the robot.py',
 'rex-images',
 'rex-the-robot-dataset',
 'rex_models',
 'rex_model_class.json',
 'rex_weight_model.063-0.8666666746139526.h5',
 'Run Rex the robot.ipynb']

In [3]:

# current working directory
execution_path = os.getcwd()

# ----------------- The Section Responsible for Downloading the Dataset ---------------------


SOURCE_PATH = r"C:\Users\lizzi\OneDrive\UNI\2019\thesis\Machine Learning Course\YWLAI\rex"
FILE_DIR = os.path.join(execution_path, "rex-the-robot-dataset")
DATASET_DIR = os.path.join(execution_path, "rex-images")
DATASET_TRAIN_DIR = os.path.join(DATASET_DIR, "train")
DATASET_TEST_DIR = os.path.join(DATASET_DIR, "test")

In [4]:






# ----------------- The Section Responsible for Training ResNet50 on the IdenProf dataset ---------------------

# Directory in which to create models
save_direc = os.path.join(os.getcwd(), 'rex_models')

# Name of model files
model_name = 'rex_weight_model.{epoch:03d}-{val_acc}.h5'

# Create Directory if it doesn't exist
if not os.path.isdir(save_direc):
    os.makedirs(save_direc)
# Join the directory with the model file
modelpath = os.path.join(save_direc, model_name)

# Checkpoint to save best model
checkpoint = ModelCheckpoint(filepath=modelpath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True,
                             save_weights_only=True,
                             period=1)


In [5]:

# Function for adjusting learning rate and saving dummy file
def lr_schedule(epoch):
    """
    Learning Rate Schedule
    """
    # Learning rate is scheduled to be reduced after 80, 120, 160, 180  epochs. Called  automatically  every
    #  epoch as part  of  callbacks  during  training.

    lr = 1e-3
    if epoch > 180:
        lr *= 1e-4
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1

    print('Learning rate: ', lr)
    return lr


lr_scheduler = LearningRateScheduler(lr_schedule)


def resnet_module(input, channel_depth, strided_pool=False):
    residual_input = input
    stride = 1

    if (strided_pool):
        stride = 2
        residual_input = Conv2D(channel_depth, kernel_size=1, strides=stride, padding="same",
                                kernel_initializer="he_normal")(residual_input)
        residual_input = BatchNormalization()(residual_input)

    input = Conv2D(int(channel_depth / 4), kernel_size=1, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(int(channel_depth / 4), kernel_size=3, strides=1, padding="same", kernel_initializer="he_normal")(
        input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(channel_depth, kernel_size=1, strides=1, padding="same", kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)

    input = add([input, residual_input])
    input = Activation("relu")(input)

    return input


def resnet_first_block_first_module(input, channel_depth):
    residual_input = input
    stride = 1

    residual_input = Conv2D(channel_depth, kernel_size=1, strides=1, padding="same", kernel_initializer="he_normal")(
        residual_input)
    residual_input = BatchNormalization()(residual_input)

    input = Conv2D(int(channel_depth / 4), kernel_size=1, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(int(channel_depth / 4), kernel_size=3, strides=stride, padding="same",
                   kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)
    input = Activation("relu")(input)

    input = Conv2D(channel_depth, kernel_size=1, strides=stride, padding="same", kernel_initializer="he_normal")(input)
    input = BatchNormalization()(input)

    input = add([input, residual_input])
    input = Activation("relu")(input)

    return input


def resnet_block(input, channel_depth, num_layers, strided_pool_first=False):
    for i in range(num_layers):
        pool = False
        if (i == 0 and strided_pool_first):
            pool = True
        input = resnet_module(input, channel_depth, strided_pool=pool)

    return input


def ResNet50(input_shape, num_classes=2):
    input_object = Input(shape=input_shape)
    layers = [3, 4, 6, 3]
    channel_depths = [256, 512, 1024, 2048]

    output = Conv2D(64, kernel_size=7, strides=2, padding="same", kernel_initializer="he_normal")(input_object)
    output = BatchNormalization()(output)
    output = Activation("relu")(output)

    output = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(output)
    output = resnet_first_block_first_module(output, channel_depths[0])

    for i in range(4):
        channel_depth = channel_depths[i]
        num_layers = layers[i]

        strided_pool_first = True
        if (i == 0):
            strided_pool_first = False
            num_layers = num_layers - 1
        output = resnet_block(output, channel_depth=channel_depth, num_layers=num_layers,
                              strided_pool_first=strided_pool_first)

    output = GlobalAvgPool2D()(output)
    output = Dense(num_classes)(output)
    output = Activation("softmax")(output)

    model = Model(inputs=input_object, outputs=output)

    return model


def train_network():
    #download_idenprof()

    #print(os.listdir(os.path.join(execution_path, "Rex the robot")))

    optimizer = keras.optimizers.Adam(lr=0.03, decay=1e-4)
    batch_size = 5
    num_classes = 2
    epochs = 100

    model = ResNet50((224, 224, 3), num_classes=num_classes)
    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    model.summary()

    print("Using real time Data Augmentation")
    train_datagen = ImageDataGenerator(
        rescale=1. / 255,
        horizontal_flip=True)

    test_datagen = ImageDataGenerator(
        rescale=1. / 255)

    train_generator = train_datagen.flow_from_directory(DATASET_TRAIN_DIR, target_size=(224, 224),
                                                        batch_size=batch_size, class_mode="categorical")
    test_generator = test_datagen.flow_from_directory(DATASET_TEST_DIR, target_size=(224, 224), batch_size=batch_size,
                                                      class_mode="categorical")

    model.fit_generator(train_generator, steps_per_epoch=int(150 / batch_size), epochs=epochs,
                        validation_data=test_generator,
                        validation_steps=int(30 / batch_size), callbacks=[checkpoint, lr_scheduler]);



In [11]:

# ----------------- The Section Responsible for Inference ---------------------
CLASS_INDEX = None

MODEL_PATH = os.path.join(execution_path, "rex_weight_model.063-0.8666666746139526.h5")
JSON_PATH = os.path.join(execution_path, "rex_model_class.json")


def preprocess_input(x):
    x *= (1. / 255)

    return x


def decode_predictions(preds, top=5, model_json=""):
    global CLASS_INDEX

    if CLASS_INDEX is None:
        CLASS_INDEX = json.load(open(model_json))
    results = []
    for pred in preds:
        top_indices = pred.argsort()[-top:][::-1]
        for i in top_indices:
            each_result = []
            each_result.append(CLASS_INDEX[str(i)])
            each_result.append(pred[i])
            results.append(each_result)

    return results


def run_inference():
    model = ResNet50(input_shape=(224, 224, 3), num_classes=2)
    model.load_weights(MODEL_PATH)

    picture = os.path.join(execution_path, "interference-image.jpg")

    image_to_predict = image.load_img(picture, target_size=(
        224, 224))
    image_to_predict = image.img_to_array(image_to_predict, data_format="channels_last")
    image_to_predict = np.expand_dims(image_to_predict, axis=0)

    image_to_predict = preprocess_input(image_to_predict)

    prediction = model.predict(x=image_to_predict, steps=1)

    predictiondata = decode_predictions(prediction, top=int(5), model_json=JSON_PATH)

    for result in predictiondata:
        print(str(result[0]), " : ", str(result[1] * 100))




In [7]:
train_network()


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 112, 112, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           batch_normalization_1[0][0]      
_____________________________________

__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 28, 28, 128)  32896       activation_10[0][0]              
__________________________________________________________________________________________________
batch_normalization_13 (BatchNo (None, 28, 28, 128)  512         conv2d_13[0][0]                  
__________________________________________________________________________________________________
activation_11 (Activation)      (None, 28, 28, 128)  0           batch_normalization_13[0][0]     
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 28, 28, 128)  147584      activation_11[0][0]              
__________________________________________________________________________________________________
batch_normalization_14 (BatchNo (None, 28, 28, 128)  512         conv2d_14[0][0]                  
__________

batch_normalization_24 (BatchNo (None, 28, 28, 512)  2048        conv2d_24[0][0]                  
__________________________________________________________________________________________________
add_7 (Add)                     (None, 28, 28, 512)  0           batch_normalization_24[0][0]     
                                                                 activation_19[0][0]              
__________________________________________________________________________________________________
activation_22 (Activation)      (None, 28, 28, 512)  0           add_7[0][0]                      
__________________________________________________________________________________________________
conv2d_26 (Conv2D)              (None, 14, 14, 256)  131328      activation_22[0][0]              
__________________________________________________________________________________________________
batch_normalization_26 (BatchNo (None, 14, 14, 256)  1024        conv2d_26[0][0]                  
__________

batch_normalization_36 (BatchNo (None, 14, 14, 256)  1024        conv2d_36[0][0]                  
__________________________________________________________________________________________________
activation_33 (Activation)      (None, 14, 14, 256)  0           batch_normalization_36[0][0]     
__________________________________________________________________________________________________
conv2d_37 (Conv2D)              (None, 14, 14, 1024) 263168      activation_33[0][0]              
__________________________________________________________________________________________________
batch_normalization_37 (BatchNo (None, 14, 14, 1024) 4096        conv2d_37[0][0]                  
__________________________________________________________________________________________________
add_11 (Add)                    (None, 14, 14, 1024) 0           batch_normalization_37[0][0]     
                                                                 activation_31[0][0]              
__________

batch_normalization_48 (BatchNo (None, 7, 7, 512)    2048        conv2d_48[0][0]                  
__________________________________________________________________________________________________
activation_44 (Activation)      (None, 7, 7, 512)    0           batch_normalization_48[0][0]     
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 7, 7, 512)    2359808     activation_44[0][0]              
__________________________________________________________________________________________________
batch_normalization_49 (BatchNo (None, 7, 7, 512)    2048        conv2d_49[0][0]                  
__________________________________________________________________________________________________
activation_45 (Activation)      (None, 7, 7, 512)    0           batch_normalization_49[0][0]     
__________________________________________________________________________________________________
conv2d_50 

...

30/30 [==============================] - ETA: 4s - loss: 0.0113 - acc: 1.000 - ETA: 4s - loss: 0.0087 - acc: 1.000 - ETA: 4s - loss: 0.0073 - acc: 1.000 - ETA: 4s - loss: 0.0084 - acc: 1.000 - ETA: 3s - loss: 0.0107 - acc: 1.000 - ETA: 3s - loss: 0.0106 - acc: 1.000 - ETA: 3s - loss: 0.0092 - acc: 1.000 - ETA: 3s - loss: 0.0091 - acc: 1.000 - ETA: 3s - loss: 0.0640 - acc: 0.955 - ETA: 3s - loss: 0.1579 - acc: 0.920 - ETA: 3s - loss: 0.1443 - acc: 0.927 - ETA: 2s - loss: 0.1325 - acc: 0.933 - ETA: 2s - loss: 0.1225 - acc: 0.938 - ETA: 2s - loss: 0.1288 - acc: 0.928 - ETA: 2s - loss: 0.1262 - acc: 0.933 - ETA: 2s - loss: 0.1220 - acc: 0.937 - ETA: 2s - loss: 0.1158 - acc: 0.941 - ETA: 1s - loss: 0.1119 - acc: 0.944 - ETA: 1s - loss: 0.1161 - acc: 0.936 - ETA: 1s - loss: 0.1183 - acc: 0.930 - ETA: 1s - loss: 0.1139 - acc: 0.933 - ETA: 1s - loss: 0.1142 - acc: 0.936 - ETA: 1s - loss: 0.1106 - acc: 0.939 - ETA: 0s - loss: 0.1065 - acc: 0.941 - ETA: 0s - loss: 0.1089 - acc: 0.944 - ETA: 0s -

30/30 [==============================] - ETA: 4s - loss: 3.7210e-04 - acc: 1.000 - ETA: 4s - loss: 2.2372e-04 - acc: 1.000 - ETA: 4s - loss: 1.6356e-04 - acc: 1.000 - ETA: 4s - loss: 6.3807e-04 - acc: 1.000 - ETA: 3s - loss: 0.0032 - acc: 1.0000    - ETA: 3s - loss: 0.0056 - acc: 1.000 - ETA: 3s - loss: 0.0476 - acc: 0.971 - ETA: 3s - loss: 0.0417 - acc: 0.975 - ETA: 3s - loss: 0.0371 - acc: 0.977 - ETA: 3s - loss: 0.0334 - acc: 0.980 - ETA: 3s - loss: 0.0304 - acc: 0.981 - ETA: 2s - loss: 0.0279 - acc: 0.983 - ETA: 2s - loss: 0.0259 - acc: 0.984 - ETA: 2s - loss: 0.0241 - acc: 0.985 - ETA: 2s - loss: 0.0225 - acc: 0.986 - ETA: 2s - loss: 0.0212 - acc: 0.987 - ETA: 2s - loss: 0.0200 - acc: 0.988 - ETA: 1s - loss: 0.0192 - acc: 0.988 - ETA: 1s - loss: 0.0182 - acc: 0.989 - ETA: 1s - loss: 0.0190 - acc: 0.990 - ETA: 1s - loss: 0.0181 - acc: 0.990 - ETA: 1s - loss: 0.0173 - acc: 0.990 - ETA: 1s - loss: 0.0166 - acc: 0.991 - ETA: 0s - loss: 0.0160 - acc: 0.991 - ETA: 0s - loss: 0.0153 - ac

30/30 [==============================] - ETA: 4s - loss: 2.4536e-04 - acc: 1.000 - ETA: 4s - loss: 0.0051 - acc: 1.0000    - ETA: 4s - loss: 0.0041 - acc: 1.000 - ETA: 4s - loss: 0.0037 - acc: 1.000 - ETA: 3s - loss: 0.0031 - acc: 1.000 - ETA: 3s - loss: 0.0026 - acc: 1.000 - ETA: 3s - loss: 0.0023 - acc: 1.000 - ETA: 3s - loss: 0.0031 - acc: 1.000 - ETA: 3s - loss: 0.0029 - acc: 1.000 - ETA: 3s - loss: 0.0027 - acc: 1.000 - ETA: 3s - loss: 0.0026 - acc: 1.000 - ETA: 2s - loss: 0.0027 - acc: 1.000 - ETA: 2s - loss: 0.0033 - acc: 1.000 - ETA: 2s - loss: 0.0031 - acc: 1.000 - ETA: 2s - loss: 0.0055 - acc: 1.000 - ETA: 2s - loss: 0.0149 - acc: 0.987 - ETA: 2s - loss: 0.0141 - acc: 0.988 - ETA: 1s - loss: 0.0135 - acc: 0.988 - ETA: 1s - loss: 0.0128 - acc: 0.989 - ETA: 1s - loss: 0.0122 - acc: 0.990 - ETA: 1s - loss: 0.0180 - acc: 0.990 - ETA: 1s - loss: 0.0173 - acc: 0.990 - ETA: 1s - loss: 0.0166 - acc: 0.991 - ETA: 0s - loss: 0.0248 - acc: 0.983 - ETA: 0s - loss: 0.0238 - acc: 0.984 - E

30/30 [==============================] - ETA: 4s - loss: 7.8453e-04 - acc: 1.000 - ETA: 4s - loss: 0.0010 - acc: 1.0000    - ETA: 4s - loss: 0.0023 - acc: 1.000 - ETA: 4s - loss: 0.0024 - acc: 1.000 - ETA: 3s - loss: 0.0026 - acc: 1.000 - ETA: 3s - loss: 0.0022 - acc: 1.000 - ETA: 3s - loss: 0.0054 - acc: 1.000 - ETA: 3s - loss: 0.0049 - acc: 1.000 - ETA: 3s - loss: 0.0046 - acc: 1.000 - ETA: 3s - loss: 0.0045 - acc: 1.000 - ETA: 3s - loss: 0.0042 - acc: 1.000 - ETA: 2s - loss: 0.0039 - acc: 1.000 - ETA: 2s - loss: 0.0041 - acc: 1.000 - ETA: 2s - loss: 0.0038 - acc: 1.000 - ETA: 2s - loss: 0.0042 - acc: 1.000 - ETA: 2s - loss: 0.0041 - acc: 1.000 - ETA: 2s - loss: 0.0038 - acc: 1.000 - ETA: 1s - loss: 0.0037 - acc: 1.000 - ETA: 1s - loss: 0.0036 - acc: 1.000 - ETA: 1s - loss: 0.0036 - acc: 1.000 - ETA: 1s - loss: 0.0035 - acc: 1.000 - ETA: 1s - loss: 0.0033 - acc: 1.000 - ETA: 1s - loss: 0.0032 - acc: 1.000 - ETA: 0s - loss: 0.0031 - acc: 1.000 - ETA: 0s - loss: 0.0030 - acc: 1.000 - E

30/30 [==============================] - ETA: 4s - loss: 2.0481e-04 - acc: 1.000 - ETA: 4s - loss: 0.0014 - acc: 1.0000    - ETA: 4s - loss: 0.0024 - acc: 1.000 - ETA: 4s - loss: 0.0018 - acc: 1.000 - ETA: 3s - loss: 0.0017 - acc: 1.000 - ETA: 3s - loss: 0.0018 - acc: 1.000 - ETA: 3s - loss: 0.0015 - acc: 1.000 - ETA: 3s - loss: 0.0019 - acc: 1.000 - ETA: 3s - loss: 0.0020 - acc: 1.000 - ETA: 3s - loss: 0.0032 - acc: 1.000 - ETA: 3s - loss: 0.0031 - acc: 1.000 - ETA: 2s - loss: 0.0029 - acc: 1.000 - ETA: 2s - loss: 0.0028 - acc: 1.000 - ETA: 2s - loss: 0.0026 - acc: 1.000 - ETA: 2s - loss: 0.0025 - acc: 1.000 - ETA: 2s - loss: 0.0025 - acc: 1.000 - ETA: 2s - loss: 0.0024 - acc: 1.000 - ETA: 1s - loss: 0.0023 - acc: 1.000 - ETA: 1s - loss: 0.0023 - acc: 1.000 - ETA: 1s - loss: 0.0022 - acc: 1.000 - ETA: 1s - loss: 0.0022 - acc: 1.000 - ETA: 1s - loss: 0.0021 - acc: 1.000 - ETA: 1s - loss: 0.0020 - acc: 1.000 - ETA: 0s - loss: 0.0020 - acc: 1.000 - ETA: 0s - loss: 0.0065 - acc: 1.000 - E

In [12]:
run_inference()

juicebox  :  99.98624324798584
chips  :  0.013753905659541488
